In [6]:
import numpy as np

X = []
Y = []

x_noise = 0.0
y_noise = 0.0

# TODO: randomly occlude large parts of the state input to simulate detection inconsistencies

model_name = "cam"

with open(f"../utils/datasets/{model_name}.csv") as f:
    for i, line in enumerate(f):
        data = line.split(",")
        x = np.array(data[2:], dtype=np.float32)
        y = np.array(data[:2], dtype=np.float32)

        x[x > 1] = 0  # track only
        y = y[1]  # steer only

        grid_size = 30
        x = x.reshape(grid_size, grid_size, 1)  # transform state back to grid

        X.append(x)
        Y.append(y)

X = np.array(X, dtype=np.float32)
Y = np.array(Y, dtype=np.float32)

print(f"Loaded {len(X)} entires")
print(f"entry {x.shape=}, {y.shape=}")

Loaded 2491 entires
entry x.shape=(30, 30, 1), y.shape=()


In [1]:
import keras
print(keras.__version__)

2.13.1


In [10]:
import keras

model = keras.models.Sequential(
    [
        keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(30, 30, 1)),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation="relu"),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(1),
    ]
)

# TODO: add normal following to all experts' ds

model.compile(
    optimizer="adam",
    loss="mean_squared_error",
)

model.fit(
    X,
    Y,
    epochs=10,
    validation_split=0.2,
    callbacks=[keras.callbacks.EarlyStopping(patience=1)],
)

model.save(f"../controllers/moe_controller/models/{model_name}-new.keras")

Epoch 1/10
63/63 [==============================] - 1s 13ms/step - loss: 0.0352 - val_loss: 0.0139
Epoch 2/10
63/63 [==============================] - 0s 8ms/step - loss: 0.0113 - val_loss: 0.0120
Epoch 3/10
63/63 [==============================] - 0s 8ms/step - loss: 0.0108 - val_loss: 0.0119
Epoch 4/10
63/63 [==============================] - 0s 7ms/step - loss: 0.0084 - val_loss: 0.0119
Epoch 5/10
63/63 [==============================] - 0s 8ms/step - loss: 0.0078 - val_loss: 0.0116
Epoch 6/10
63/63 [==============================] - 0s 7ms/step - loss: 0.0074 - val_loss: 0.0109
Epoch 7/10
63/63 [==============================] - 0s 8ms/step - loss: 0.0060 - val_loss: 0.0113


In [25]:
# time inference speed
import time

start = time.time()
model(X[0:100])
end = time.time()
avg = (end - start) / 100
print(f"avg inference time: {avg*1000:.2f}ms")

avg inference time: 0.11ms
